# Lab 01: SoftwareIntro
The purpose of this homework is to guide you through opening the programs you've downloaded through Neurodebian and getting familiar with some of their basic functions to understand the basics of how images are stored and how we can interact with them.


## Preparation 

### Download example data

We will first download some practice MRI data from open science framework. While this is not a common outlet for scientists to share their MR imaging datasets, this is a useful repository for files that are larger than we would want to put on github and are in a simplified storage format for teaching and demonstrations like this one.


In [ ]:
%%bash

#the following bash commands will allow us to download the data using command line tools
#this may take a little time, it is done when the astericks goes away
wget -O lab01_images.tar.gz https://osf.io/bprq5/download/
tar -xvf lab01_images.tar.gz

In [ ]:
%%bash
pwd; ls  #tell me present working directoy; then list contents of it

In [ ]:
%%bash
ls ./osfshare  #list contents within osfshare

### Preview the data directly in jupyter

We'll first get used to terms about the spatial orientation of an MR image. We'll use a viewer that is part of nipy, which is a collection of neuroimaging tools written in the python programming language. 

More information about the NiftiWidget can be found here: http://nipy.org/niwidgets/examples.html. 

__Import an example image that comes with the package__

In [ ]:
#import the widget from the niwidgets package
from niwidgets import NiftiWidget   

#import example data that comes with niwidgets
from niwidgets import examplet1   

#commands that open the image in the widget
test_widget = NiftiWidget(examplet1)
test_widget.nifti_plotter()

__Import our own image__ <br>
What are some similarities and differences?

In [ ]:
#give our anatomical image a variable name
mydata='./osfshare/sub-97-T1w_defaced.nii.gz'

#what exactly does each do below
mydata_widget = NiftiWidget(mydata)
mydata_widget.nifti_plotter()

The images we have been viewing are high-resolution images using [T1 contrast](http://www.med.harvard.edu/aanlib/basicsMR.html), and the image format is [nifti](https://brainder.org/2012/09/23/the-nifti-file-format/). As you see above, the image is 3-dimensional and can be opened in 3 orthogonal views simultaneously; coronal, sagittal and axial views.The viewer above is useful for getting familiar with the x,y,and z "planes" of the image. Every 3-dimensional pixel in the image (called a voxel) has a unique x,y,z coordinate.  <br>
<br>
It is also common to refer to different parts of the image by their location in neuroanatomical terms: L, R, A, P, S, I (Left, Right, Anterior=front, Posterior=back, Superior=top, Inferior=bottom). We will see that some viewers will show us these labels and "Left" in the image is not always the "Left" part of the head! <br>
<br>
It may be obvious, but also note that in these images we cannot see at the level of single neurons. The anatomy we focus on with MRI is from a "macroscopic" view. 


## FSL package viewer

A very common neuroimaging package is FSL. FSL has viewer that we can open in the bash terminal, and it has some very helpful features for understanding more about images.

__We'll first see how to:__ <br>
<br>
__(a) learn basic information about the image attributes__ <br> 
-What kind of image is this <br>
-What is the voxel size <br>
-Others? <br>
<br>
__(b) look at the image histogram.__ <br>
An image histogram shows the distribution of all the intensity values that make up the image. Looking at these is helpful to see what intensities are generally “brain” versus background and to get comfortable with the idea that images are a collection of voxels with different intensity values (e.g., see Handbook Figure 2.1, p14). The intensity values mean something about what tissue we *think* we’re seeing based on the physics of MRI. It is also helpful to remember that the basis of most image processing tools is signal processing and statistics on these intensity values to improve the detection of the "signal" we're interested in (e.g., brain structure or activity) relative to noise (e.g., motion). <br>
-What is the range of values in our sub-97-T1w_defaced.nii.gz image <br>
-Is the histogram multi-modal? What would that tell us?  <br>
-What are the typical values for gray matter, white matter, and CSF? <br>
<br>
__(c) look at our functional image and its timeseries.__ <br>
-Compare and contrast the image intensities of white matter and gray matter from the structural with the functional image. <br>
-Show the image timeseries <br>
-Show the movie of volumes over time, how many dimensions is the file? <br>
-Compare the spatial resolution of the structural and functional <br>
-Can we overlay the functional on top of the structural image? <br>

In [ ]:
%%bash

#the viewer version we'll use is called fslview
#open in the command line by typing fslview and hitting enter
fslview

## Image meta-data

In addition to the information in the image, another important aspect of data with the images is their meta-data (Handbook, p14). This information is often stored in a [header](http://nipy.org/nibabel/nibabel_images.html#the-image-header) that is tagged onto the image file itself or as a separate file. For nifti images the header information is kept tagged onto the image itself, so it is a handy to know how to look at the header for images on the command line. We'll see how to do this with fsl and nibabel below.

In [ ]:
import nibabel as nb
import os

#below is a different way to assign variable names to directories and files
datadir="./osfshare/"
anat_file=os.path.join(datadir,"sub-97-T1w_brain.nii.gz")
anat_file
anat_image=nb.load(anat_file)
func_file=os.path.join(datadir,"sub-97_task-flanker_bold.nii.gz")
func_file
func_image=nb.load(func_file)

In [ ]:
#look at metadata for dimensions of our image, compare to what we saw with fslview
anat_image.shape  

In [ ]:
#header
anat_image.header  #this is the header object
print(anat_image.header)  #print it to the screen

In [ ]:
%%bash

#fsl also makes it easy to see header information on the command line with fslhd
fslhd ./osfshare/sub-97-T1w_brain.nii.gz

## Coordinate systems and affines

As described in the Handbook (Ch 2), images are arrays of numbers. We also need to consider how that image is positioned in space, and we'll get a more hands-on look at this information here. Many of these examples are used from this excellent reference on [arrays](http://nipy.org/nibabel/coordinate_systems.html). In particular, be sure we discuss the [naming of reference spaces](http://nipy.org/nibabel/coordinate_systems.html#naming-reference-spaces) in relation to what we see below, and we'll need to understand the qform and sform information in the headers as described more [here](https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/Orientation%20Explained).

In [ ]:
#let's plot the image with the voxel coordinate system viewable
import pylab as plt

#the get_data() function allows us to access the actual data in the image object. This will load the image data into memory. 
data=anat_image.get_data()

#try these different commands, what is happening here?

plt.imshow(data[:,:,89],cmap="gray")
#plt.imshow(data[160,:,:],cmap="gray")
#plt.imshow(data[:,104,:],cmap="gray")


In [ ]:
%%bash
#now let's use fslview to visually compare coordinates for the image in voxel space (voxels) and scanner space (mm) 
#do you remember how to open fslview from the command line?

Now let's look at using matrices to represent the spatial transformations between different image "spaces." Review this [explanation](http://nipy.org/nibabel/coordinate_systems.html#the-affine-matrix-as-a-transformation-between-spaces) which complements and extends the information in the Handbook Box 2.3.1.

In [ ]:
#look at affine array for our anatomical image
#this  maps the voxel coordinates to scanner coordinates
anat_image.affine

In [ ]:
%%bash

#let's change the orientation of the image and see how it affects the affine matrix
#the following fsl program allows you to change the orientation of your image
#you can type the name of the program into the bash terminal to see the usage:
fslswapdim

In [ ]:
%%bash

#let's swap the dimension of our image across the y axis and see what happens
fslswapdim ./osfshare/sub-97-T1w_brain.nii.gz -x y z ./osfshare/sub-97-T1w_brain_LPI.nii.gz

In [ ]:
##is the change reflected in the header?  how can you check?

In [ ]:
%%bash
#compare images by loading them all together
fslview ./osfshare/sub-97-T1w_brain_LPI.nii.gz ./osfshare/sub-97-T1w_brain.nii.gz ./osfshare/sub-97-T1w_defaced.nii.gz

In [ ]:
%%bash
#a useful tool..
fslorient

In [ ]:
%%bash
fslorient -swaporient ./osfshare/sub-97-T1w_brain_LPI.nii.gz

In [ ]:
%%bash 
#now look, what changed?
fslhd ./osfshare/sub-97-T1w_brain_LPI.nii.gz

In [ ]:
%%bash
fslview ./osfshare/sub-97-T1w_brain_LPI.nii.gz

<br>
## Practice problems 

Below we list some practice problems to review what we've learned.

_1)_ Extract the dimensions of the functional image using nipype commands as we did for the anatomical above. Show your work below. Then in the markdown cell explain what the dimensions are and what they mean in words. (e.g., "The dimensions of each volume are --- and the were --- volumes in the timeseries"). 

In [ ]:
#code here

Answer in words here

_2)_ Compare the image intensity values for the structural and functional image for the same brain tissue types. What tissues are brightest in the T1-weighted structural image? How does this compare to the intensities in the T2-weighted functional image. Use a code cell below to give an example from a given coordinate, and explain your answer in words in the following markdown cell. It may also help to refer to Ch2 in our Handbook text. 

In [ ]:
#code here

Answer in words here

_3)_ Use what we learned about affine representations for images to explain: <br>
<br>
_a)_ Why is there a *-1* in the the [1,1] affine matrix element for our T1-weighted image and how does it change when we reorient the image? It may help to re-read [this page](https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/Orientation%20Explained). These operations are used often and can be buried in early data reconstruction scripts so you want to understand and know how to check this information!<br>


Answer in words here

_b)_ What do the values in column 4 of the affine matrix refer to. Include what kind of transformation this is on the image.

Answer in words here